# Topic Guardrail

![Solution Architecture](./images/w_topical.png)

## Topical Rail - Demo
Developing chatbots that stay on topic can be challenging. The key is to build the bot to be knowledgeable about specific subjects.

In this example, we'll look at some ways to keep a chatbot on topic, including:

- Constructing the chatbot knowledge to be focused on particular topics and conversations 
- Providing a quick overview of how to launch a chatbot with mechanisms to keep it on a single topic
- Demonstrating dialogues that illustrate the chatbot sticking to certain subjects and avoiding others

The goal is to guide the chatbot to stay on target when users ask questions, rather than letting the conversation drift aimlessly. By designing the chatbot well and giving it the right scopes of knowledge, we can create more useful and effective conversational agents.  



### How does NeMo Guardrails works for topical identification (in a nutshell)
Topical rail ensures AI responses stay within the predefined topic, and prevents off-topic conversations that has no business value.
1. As part of Guardrails configuration, you define different conversation flow and how Guardrails treat them.
2. You provide example input texts for each flow.
3. When the user sends text, Guardrails intercepts it, and tries to understand which flow it maps to (Embedding based similarity search vs each flow's examples texts). 
4. With the flow established Guardrails carries out the flow (e.g., let's the LLM respond, or replies back the conversation is off-topic).

In [11]:
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [12]:
import json
import boto3
from utils import bedrock

# This helper function encompasses the process of initializing NeMo Guardrails and generating Rails based on a specified configuration.
from nemoguardrails import LLMRails, RailsConfig

# BedrockModels is a "Singleton" class which initializes the necessary models for the notebook.
from models import BedrockModels

# This creates a pywidget chat window for testing
from models import ChatComponent

## Initalize bedrock client

In [13]:
for path in sys.path:
    if "guardrails" in path.lower():
        sys.path.append(os.path.join(path, 'NeMo'))
        break

from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)


Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Define function to create a LLMRails instance with Claudev2 as LLM and a certain rail configuration

In [14]:
# This helper function encapsulates the necessary steps to bootstrap
# NeMo Guardrails and returns Rails based on a given configuration.
def bootstrap_bedrock_nemo_guardrails(rail_config_path: str) -> LLMRails:

    #1. initialize rails config
    config = RailsConfig.from_path(f"NeMo/rails/{rail_config_path}/config")

    # initialize bedrock models
    # you can pass model id as string or use the default model id 'anthropic.claude-v2'
    bedrock_models = BedrockModels
    bedrock_models.init_bedrock_client(boto3_bedrock)
    bedrock_models.init_llm('anthropic.claude-v2')

    # 2. bootstraps NeMo Guardrails with the necessary resources
    app = LLMRails(config=config,llm=bedrock_models.llm,  verbose=True)
    
    return app


## rail configuration

To view the full `on-topic.co` configuration content, run the following cell:

In [15]:
!cat ./NeMo/rails/topical/config/on-topic.co




define user ask machine learning and public sector
    "What challenges are faced in data ingestion and preparation for ML in public sector?"
    "How is model training and tuning particularly challenging for public sector organizations?"
    "What hurdles exist in integrating ML into business operations (MLOps) within the public sector?"
    "How is management and governance of ML projects handled in the public sector?"
    "What security and compliance challenges are encountered in implementing ML projects?"
    "How do cost factors impact the implementation of ML projects in the public sector?"
    "What concerns surround bias and explainability in ML models within public sector organizations?"
    "How do public sector organizations ensure ethical considerations in ML implementations?"
    "What steps are needed to ensure data is properly cataloged and organized for ML projects?"
    "How do regulatory frameworks impact ML implementation in the public sector?"

define bot answer 

To view the full `off-topic.co` configuration content, run the following cell:

In [16]:
!cat ./NeMo/rails/topical/config/off-topic.co


define user ask politics
    "why doesn't the X party care about Y?"
    "what are your political views?"
    "who should I vote for?"
    "who should run for president?"
    "How are political campaigns strategized?"
    "What is the significance of debates in a political campaign?"
    "How are political advertisements regulated?"
    "How do political endorsements affect a campaign?"
    "What is the difference between a caucus and a primary?"
    "What are the functions of different political offices?"
    "How do international relations affect domestic politics?"
    "What is the process of impeachment?"
    "How are election dates determined?"
    "What are the roles of the different branches of government?"
    "What is the importance of checks and balances in government?"
    "How do midterm elections differ from presidential elections?"
    "What is the significance of a swing state?"
    "What are the major political ideologies and how do they differ?"
    "What are the roles

## Test the rail

In [17]:
# Bootstrap Guardrails with topical configuration
rails = bootstrap_bedrock_nemo_guardrails('topical')

Entered verbose mode.

Human conversations vector store index loaded from disk.

Assistant conversations vector store index loaded from disk.

NeMo Conversations Flows vector store index loaded from disk.

KnowledgeBase vector store index loaded from disk.


In [18]:
import nest_asyncio
nest_asyncio.apply()

In [19]:
response = await rails.generate_async(messages=[
    {
    "role": "user",
    "content": "Who should i vote for?"}])

print_ww(response["content"])

Event UtteranceUserActionFinished {'final_transcript': 'Who should i vote for?'}
Event StartInternalSystemAction {'uid': '4c8b8770-5219-4db6-8070-4f6a57ff5c84', 'event_created_at': '2024-02-16T23:25:48.326757+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': '00f3d9ca-113d-43fb-8179-60958ca56177', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': '20db7309-644d-4d0a-919f-f5ad575ef0e6', 'event_created_at': '2024-02-16T23:25:48.327439+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'Who should i vote for?'}
Event StartInternalSystemAction {'uid': '31a4e0ee-fad2-4341-9abc-f59985673b4e', 'event_created_at': '2024-02-16T23:25:48.328355+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '4d615058-9a8d-496f-bdd9-40e6fc4fbbeb', 'is_sys

## Experiment with the Chat Interface and Review Sample Questions Ahead
Please feel free to interact with the chat below. Once completed, proceed to the next block to examine sample questions.

#### You can also try these On-Topic and Off-Topic sample questions

#### <font color="#4ec8b0">On-Topic questions:</font>

* what are the Government, education, and nonprofit organizations challenges when implementing ML programs to accomplish their objectives?
* what would be the most important thing to do to overcome the first challenge?
* What are the primary benefits and potential impact of implementing machine learning technologies within public sector operations?

#### <font color="#c84e4e">Off-topic questions:</font>

* Who should i vote for?
* Give me a few tips on how to start a new garden
* What are the primary considerations when planning a long-distance hiking trip? 
* What are the benefits and drawbacks of adopting a gluten-free diet?

<div class="alert alert-block alert-warning"><b>Info:</b> The markup below serves solely to enhance the UI and design of the chat component, without adding any additional functionality.</div>

In [11]:
%%html

<style>
    .chat-message-content-xmp{ white-space:pre-wrap; word-wrap:break-word; }

    .chat-message-left,
    .chat-message-right {
        display: flex;
        flex-shrink: 0;
        margin-right: 1rem;
    }

    .chat-message-left {
        margin-right: auto
        margin-left: 1rem;
    }

    .chat-message-right {
        flex-direction: row-reverse;
        margin-left: auto
    }
</style>


In [12]:
topical_chat = ChatComponent(llm=topical_llm)
topical_chat.render()


Box(children=(Image(value=b'GIF89a\xc8\x00\xc8\x00\xf7\x00\x00;Ch\x83\x90\xb7\xcf\xdc\xe8\xda\xec\xf1\xf1\xf2\…